<a href="https://colab.research.google.com/github/voirinprof/python-geo-colab/blob/main/d%C3%A9buter_avec_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutoriel : Découverte de Google Earth Engine et Google Colab

Vous allez apprendre à utiliser deux outils puissants :
- **Google Colab** : Un environnement gratuit pour écrire et exécuter du code Python dans le cloud.
- **Google Earth Engine (GEE)** : Une plateforme pour analyser des données géospatiales (images satellites, données climatiques, etc.).

### Objectif

1. Configurer notre environnement.
2. Charger une image satellite avec GEE.
3. Calculer un indice de végétation (NDVI).
4. Visualiser les résultats sur une carte interactive.
5. Exporter dans Google Drive


Prérequis : Un compte Google (pour Colab, GEE et Google Drive).

## Installer GEE dans Colab

In [1]:
# Installer et configurer GEE
!pip install earthengine-api  # Installe l'API GEE

## Importer les libs utiles

In [3]:
import ee  # Importe la bibliothèque GEE
import folium  # Pour afficher des cartes interactives

# Authentification (exécutez cette cellule et suivez les instructions)
ee.Authenticate()  # Ouvre une fenêtre pour se connecter à votre compte Google
ee.Initialize(project="gmq580")  # Initialise GEE avec vos identifiants

# il peut être nécessaire de créer un projet dans Google Cloud Console

print("GEE est prêt à être utilisé !")

GEE est prêt à être utilisé !


## Débuter votre analyse

Définir une zone d’intérêt

In [4]:
# On va utiliser la localisation de Sherbrooke
region = ee.Geometry.Point([-71.888351, 45.404476])  # Centre sur Sherbrooke
area = region.buffer(10000)  # Crée une zone tampon de 10 km autour du point

Charger des données satellites et calculer le NDVI

In [6]:
# Charger une collection Sentinel-2 (images satellites)
# on limite à notre zone d'étude
# on limite une période temporelle
# on souhaite les images avec peu de nuages

# de plus, on va combiner les images en une seule image (médiane des images)
sentinel = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterBounds(area) \
            .filterDate('2024-06-01', '2024-08-31') \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .median()

# Calculer le NDVI (indice de végétation)
ndvi = sentinel.normalizedDifference(['B8', 'B4']).rename('NDVI')
# on utilise la méthode GEE pour faire le calcul

print("NDVI calculé avec succès !")

NDVI calculé avec succès !


Visualiser sur une carte

In [8]:
# Fonction pour ajouter des couches GEE à Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Ajouter la méthode à Folium
folium.Map.add_ee_layer = add_ee_layer

# Créer une carte centrée sur la région
m = folium.Map(location=[45.404476, -71.888351], zoom_start=10)

# Paramètres de visualisation pour le NDVI
ndvi_params = {'min': 0, 'max': 1, 'palette': ['red', 'yellow', 'green']}

# Ajouter le NDVI à la carte
m.add_ee_layer(ndvi, ndvi_params, 'NDVI')

# Afficher la carte
display(m)

Analyse supplémentaire

In [9]:
# Calculer des statistiques sur le NDVI
stats = ndvi.reduceRegion(
    reducer=ee.Reducer.mean().combine(ee.Reducer.minMax(), None, True),
    geometry=area,
    scale=10
)

# Afficher les résultats
stats_dict = stats.getInfo()
print("Statistiques du NDVI dans la zone :")
print(f"Moyenne : {stats_dict['NDVI_mean']:.3f}")
print(f"Minimum : {stats_dict['NDVI_min']:.3f}")
print(f"Maximum : {stats_dict['NDVI_max']:.3f}")

Statistiques du NDVI dans la zone :
Moyenne : 0.632
Minimum : -0.326
Maximum : 0.860


Extra : on peut afficher avec GEEMAP aussi

In [11]:

import geemap


# Display the image using geemap
Map = geemap.Map()
Map.centerObject(region, 10)
Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Sentinel-2')

ndvi_params = {
    'min': 0,
    'max': 1,
    'palette': ['red', 'yellow', 'green']
}

# Ajouter le NDVI à la carte
Map.addLayer(ndvi, ndvi_params, 'NDVI')
Map



Map(center=[45.404476, -71.888351], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

## Sauvegarder les résultats

La méthode Export permet de sauvegarder les résultats dans Google Drive ou un Google Bucket (dans le cloud).

In [12]:
# Définir les paramètres d'exportation
export_params = {
    'image': ndvi,  # L'image à exporter (ici le NDVI)
    'description': 'NDVI_Sherbrooke_2024',  # Nom du fichier (sans espaces ni caractères spéciaux)
    'folder': 'GEE_Exports',  # Dossier dans Google Drive (sera créé s'il n'existe pas)
    'scale': 10,  # Résolution en mètres (Sentinel-2 a une résolution native de 10 m)
    'region': area,  # Zone à exporter
    'maxPixels': 1e9  # Limite de pixels (augmentez si nécessaire)
}

# Lancer l'exportation vers Google Drive
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

print("Exportation lancée ! Vérifiez votre Google Drive dans quelques minutes (dossier 'GEE_Exports').")

Exportation lancée ! Vérifiez votre Google Drive dans quelques minutes (dossier 'GEE_Exports').


In [13]:
# Vérifier l'état de la tâche
import time

# Boucle pour vérifier l'état toutes les 10 secondes
for _ in range(10):  # Vérifie pendant 100 secondes max
    status = task.status()['state']
    print(f"État de l'exportation : {status}")
    if status in ['COMPLETED', 'FAILED']:
        break
    time.sleep(10)

if status == 'COMPLETED':
    print("Exportation terminée ! Fichier disponible dans Google Drive.")
elif status == 'FAILED':
    print("Erreur lors de l'exportation. Vérifiez les paramètres.")
else:
    print("L'exportation est en cours. Cela peut prendre plus de temps.")

État de l'exportation : RUNNING
État de l'exportation : RUNNING
État de l'exportation : RUNNING
État de l'exportation : RUNNING
État de l'exportation : RUNNING
État de l'exportation : RUNNING
État de l'exportation : RUNNING
État de l'exportation : COMPLETED
Exportation terminée ! Fichier disponible dans Google Drive.
